In [2]:
from setfit import SetFitModel
import torch
import os
import pandas as pd

/mnt/c/Users/Sergi/Documents/BioTermCategorizer/.env_biotermcategorizer/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
path = '/mnt/c/Users/Sergi/Desktop/BSC/modelos_entrenados/SetFit/noparents_sp'
model = SetFitModel.from_pretrained(path)
all_labels = ['ACTIVIDAD', 'COMUNIDAD', 'DEPARTAMENTO', 'ENFERMEDAD', 'FAC_GEN', 'FAC_NOM', 'FARMACO', 'GEO_GEN', 'GEO_NOM', 'GPE_GEN', 'GPE_NOM', 'HUMAN', 'IDIOMA', 'MORFOLOGIA_NEOPLASIA', 'NO_CATEGORY', 'PROCEDIMIENTO', 'PROFESION', 'SINTOMA', 'SITUACION_LABORAL', 'SPECIES', 'TRANSPORTE']

/mnt/c/Users/Sergi/Documents/BioTermCategorizer/.env_biotermcategorizer/lib/python3.10/site-packages/sklearn/base.py:347: InconsistentVersionWarning: Trying to unpickle estimator LogisticRegression from version 1.2.2 when using version 1.3.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/mnt/c/Users/Sergi/Documents/BioTermCategorizer/.env_biotermcategorizer/lib/python3.10/site-packages/sklearn/base.py:347: InconsistentVersionWarning: Trying to unpickle estimator MultiOutputClassifier from version 1.2.2 when using version 1.3.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [13]:
def compute_predictions(mention):
    """
    Computes label predictions for a given mention.

    Parameters:
    mention (str): The input mention for prediction.

    Returns:
    list: List of filtered labels considering the given threshold and the maximum labels.
    """
    embeddings = model.model_body.encode([mention], normalize_embeddings=model.normalize_embeddings, convert_to_tensor=True)
    predicts = model.model_head.predict_proba(embeddings)
    predscores = {all_labels[i]: arr[:,1].tolist()[0] for i, arr in enumerate(predicts)}
    top_n_labels = sorted(predscores, key=predscores.get, reverse=True)[:2]
    filtered_labels = [label for label in top_n_labels if predscores[label] > 0.5]
    return filtered_labels

In [14]:
directorio_origen = '/mnt/c/Users/Sergi/Desktop/BSC/resultados_extraccion'
directorio_destino = '/mnt/c/Users/Sergi/Desktop/BSC/resultados_extraccion_clasificados'

dataframes = []
archivos_tsv = [archivo for archivo in os.listdir(directorio_origen) if archivo.endswith('.tsv')]

for archivo in archivos_tsv:
    preds = []
    ruta_archivo = os.path.join(directorio_origen, archivo)
    df = pd.read_csv(ruta_archivo, sep='\t')
    mentions = df["text"].tolist()
    for mention in mentions:
        pred = compute_predictions(mention)
        preds.append(pred)
    df['pred_label'] = preds
    ruta_destino = os.path.join(directorio_destino, archivo)
    df.to_csv(ruta_destino, sep='\t', index=False)
    print("Done")

print("Archivos TSV guardados en el nuevo directorio.")

Done
Done
Done
Done


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:13                                                                                   │
│                                                                                                  │
│   10 │   df = pd.read_csv(ruta_archivo, sep='\t')                                                │
│   11 │   mentions = df["text"].tolist()                                                          │
│   12 │   for mention in mentions:                                                                │
│ ❱ 13 │   │   pred = compute_predictions(mention)                                                 │
│   14 │   │   preds.append(pred)                                                                  │
│   15 │   df['pred_label'] = preds                                                                │
│   16 │   ruta_destino = os.path.join(directorio_destino, archivo)                                │
│                                                                                                  │
│ in compute_predictions:11                                                                        │
│                                                                                                  │
│    8 │   Returns:                                                                                │
│    9 │   list: List of filtered labels considering the given threshold and the maximum labels    │
│   10 │   """                                                                                     │
│ ❱ 11 │   embeddings = model.model_body.encode([mention], normalize_embeddings=model.normalize    │
│   12 │   predicts = model.model_head.predict_proba(embeddings)                                   │
│   13 │   predscores = {all_labels[i]: arr[:,1].tolist()[0] for i, arr in enumerate(predicts)}    │
│   14 │   top_n_labels = sorted(predscores, key=predscores.get, reverse=True)[:2]                 │
│                                                                                                  │
│ /mnt/c/Users/Sergi/Documents/BioTermCategorizer/.env_biotermcategorizer/lib/python3.10/site-pack │
│ ages/sentence_transformers/SentenceTransformer.py:165 in encode                                  │
│                                                                                                  │
│   162 │   │   │   features = batch_to_device(features, device)                                   │
│   163 │   │   │                                                                                  │
│   164 │   │   │   with torch.no_grad():                                                          │
│ ❱ 165 │   │   │   │   out_features = self.forward(features)                                      │
│   166 │   │   │   │                                                                              │
│   167 │   │   │   │   if output_value == 'token_embeddings':                                     │
│   168 │   │   │   │   │   embeddings = []                                                        │
│                                                                                                  │
│ /mnt/c/Users/Sergi/Documents/BioTermCategorizer/.env_biotermcategorizer/lib/python3.10/site-pack │
│ ages/torch/nn/modules/container.py:217 in forward                                                │
│                                                                                                  │
│   214 │   # with Any as TorchScript expects a more precise type                                  │
│   215 │   def forward(self, input):                                                              │
│   216 │   │   for module in self:                                                                │
│ ❱ 217 │   │   │   input = module(input)                                                          │
│   218 │   │   return input                                 